In [80]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import logging
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, naive_bayes, svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras import layers
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize

# Importing the cleaned data set with attribute name-category

In [81]:
c=pd.read_csv('c.csv')
y=c['attribute_value']

In [82]:
d['attribute_value']

0            top
1            top
2       onepiece
3       onepiece
4         bottom
          ...   
6775      bottom
6776      bottom
6777      bottom
6778      bottom
6779         top
Name: attribute_value, Length: 6780, dtype: object

In [83]:
df_cat=c[['product_id','description','details','name','brand_category']].reset_index().drop(columns='index')

In [84]:
y=c['attribute_value']

In [86]:
import gensim
wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

# Best Model

In [126]:
def predict_cat(brand,description,brand_category,wv):
    import warnings
    warnings.filterwarnings("ignore")
    c=pd.read_csv('c.csv')
    df_cat=c[['product_id','description','details','name','brand_category']].reset_index().drop(columns='index')
    y=c['attribute_value']
    #Clean the query
    def replace(word):
        word=word.replace('/',' ')
        new=re.sub('[^a-zA-Z]', ' ',word)
        p=[]
        for k in new.split():
            separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
            new = separated.lower()
            p.append(new)
        ls=WordNetLemmatizer()
        new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
        new = ' '.join(new)
        return new
    brand=replace(brand)
    description=replace(description)
    brand_category=replace(brand_category)
    import gensim
    from gensim.models import Word2Vec
    from sklearn.neural_network import MLPClassifier
    from sklearn.model_selection import train_test_split
    #wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
    wv.init_sims(replace=True)
    def word_averaging(wv, words):
        all_words, mean = set(), []

        for word in words:
            if isinstance(word, np.ndarray):
                mean.append(word)
            elif word in wv.vocab:
                mean.append(wv.syn0norm[wv.vocab[word].index])
                all_words.add(wv.vocab[word].index)

        if not mean:
            logging.warning("cannot compute similarity with no input %s", words)
            # FIXME: remove these examples in pre-processing
            return np.zeros(wv.vector_size,)

        mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
        return mean

    def  word_averaging_list(wv, text_list):
        return np.vstack([word_averaging(wv, post) for post in text_list ])
    def w2v_tokenize_text(text):
        tokens = []
        for sent in nltk.sent_tokenize(text, language='english'):
            for word in nltk.word_tokenize(sent, language='english'):
                if len(word) < 2:
                    continue
                tokens.append(word)
        return tokens
    names=['description','brand_category','details']
    d=pd.DataFrame()
    
    brand=w2v_tokenize_text(brand)
    description=w2v_tokenize_text(description)
    brand_category=w2v_tokenize_text(brand_category)
    
    brand=word_averaging(wv,brand).reshape(1,-1)
    description=word_averaging(wv,description).reshape(1,-1)
    brand_category=word_averaging(wv,brand_category).reshape(1,-1)
    X4=np.concatenate((brand,description,brand_category),axis=1)

    for i in names:
        X_new = df_cat.apply(lambda r: w2v_tokenize_text(r[i]),axis=1).values
        X_word_average = word_averaging_list(wv,X_new)
        X_array=pd.DataFrame(X_word_average)
        d=pd.concat([d,X_array],axis=1)

    nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100,50)).fit(d, y)
    return nn.predict(X4)

In [127]:
desc="blush linen button fastenings along front 100% linen, lining: 100%, cotton dry clean designer color: shell imported"

In [128]:
brand='Zimmermann'

In [129]:
brand_cat='clothing/jumpsuit/full length'

In [130]:
predict_cat(desc,brand,brand_cat,wv)

array(['onepiece'], dtype='<U19')

In [553]:
Q=brand+' '+brand_cat+' '+desc
X2=w2v_tokenize_text(brand)
X3=word_averaging_list(wv,X2)